In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import *

In [0]:
orders_schema = "order_id long,customer_id long,customer_fname string,customer_lname string,city string,state string,pincode long,line_items array<struct<order_item_id: long,order_item_product_id: long,order_item_quantity: long,order_item_product_price: float,order_item_subtotal: float>>"

In [0]:
spark.sql("DROP TABLE IF EXISTS orders_final_result")

Out[4]: DataFrame[]

In [0]:
dbutils.fs.rm("dbfs:/checkpointdir108",True)
dbutils.fs.rm("dbfs:/FileStore/streaming_input/input1",True)

Out[5]: True

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/streaming_input/input1")

Out[6]: True

In [0]:
orders_df = spark \
.readStream \
.format("json") \
.schema(orders_schema) \
.option("path","dbfs:/FileStore/streaming_input/input1") \
.load()

In [0]:
orders_df.createOrReplaceTempView("orders")


In [0]:
exploded_orders = spark.sql("""select order_id,customer_id,city,state,
pincode,explode(line_items) lines from orders""")

In [0]:
exploded_orders.createOrReplaceTempView("exploded_orders")

In [0]:
flattened_orders = spark.sql("""select order_id, customer_id, city, state, pincode, 
lines.order_item_id as item_id, lines.order_item_product_id as product_id,
lines.order_item_quantity as quantity,lines.order_item_product_price as price,
lines.order_item_subtotal as subtotal from exploded_orders""")

In [0]:
flattened_orders.createOrReplaceTempView("orders_flattened")

In [0]:
aggregated_orders = spark.sql("""select customer_id, approx_count_distinct(order_id) as orders_placed, 
count(item_id) as products_purchased,sum(subtotal) as amount_spent 
from orders_flattened group by customer_id""")

In [0]:
def myfunction(orders_result,batch_id):
    orders_result.createOrReplaceTempView("orders_result")
    merge_statement = """merge into orders_final_result t using orders_result s
    on t.customer_id == s.customer_id
    when matched then
    update set t.products_purchased = s.products_purchased, t.orders_placed = s.orders_placed,
    t.amount_spent = s.amount_spent
    when not matched then
    insert *
    """

    orders_result._jdf.sparkSession().sql(merge_statement)



In [0]:
streaming_query = aggregated_orders \
.writeStream \
.format("delta") \
.outputMode("update") \
.option("checkpointLocation","checkpointdir108") \
.foreachBatch(myfunction) \
.start()


In [0]:
spark.sql("create table orders_final_result (customer_id long, orders_placed long, products_purchased long, amount_spent float)")

Out[16]: DataFrame[]

In [0]:
spark.sql("select * from orders_final_result").show()

+-----------+-------------+------------------+------------+
|customer_id|orders_placed|products_purchased|amount_spent|
+-----------+-------------+------------------+------------+
|      11599|            1|                 4|   809.97003|
|       6272|            3|                 8|   1709.8401|
|       8355|            5|                13|     2669.62|
|      10280|            3|                10|     2659.76|
|       5882|            3|                 5|   674.91003|
|       9557|            8|                28|     7782.25|
+-----------+-------------+------------------+------------+



In [0]:
# streaming_query.